In [1]:
# importing libraries 
import pandas as pd 
import numpy as np # importing libraries 
import torch
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import matplotlib.pyplot as plt 

In [12]:
df = pd.read_csv('selected_articles.csv')
df.drop(columns=['Unnamed: 0'],inplace=True,axis=1)
df.head()

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc,gender,caption
0,108775015,Strap top,Vest top,Garment Upper body,Solid,Black,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,womens,"womens Solid Black Vest top, Jersey top with n..."
1,108775044,Strap top,Vest top,Garment Upper body,Solid,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,womens,"womens Solid White Vest top, Jersey top with n..."
2,108775051,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,womens,"womens Stripe Off White Vest top, Jersey top w..."
3,111586001,Shape Up 30 den 1p Tights,Leggings,Garment Lower body,Solid,Black,Tights basic,Lingeries/Tights,Ladieswear,"Womens Nightwear, Socks & Tigh",Socks and Tights,Tights with built-in support to lift the botto...,womens,"womens Solid Black Leggings, Tights with built..."
4,116379047,Frugan longsleeve,Top,Garment Upper body,Solid,Dark Blue,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Fitted top in soft stretch jersey with a wide ...,womens,"womens Solid Dark Blue Top, Fitted top in soft..."


In [13]:
df.columns

Index(['article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name', 'department_name',
       'index_name', 'index_group_name', 'section_name', 'garment_group_name',
       'detail_desc', 'gender', 'caption'],
      dtype='object')

# Taking a Sample dataset to check #

In [15]:
# del(df)

In [14]:
df_sample = df.iloc[:,[0,-1]]
df_sample.head(4)

,article_id,caption
0,108775015,"womens Solid Black Vest top, Jersey top with n..."
1,108775044,"womens Solid White Vest top, Jersey top with n..."
2,108775051,"womens Stripe Off White Vest top, Jersey top w..."
3,111586001,"womens Solid Black Leggings, Tights with built..."


In [16]:
df_sample['article_id'] = df_sample['article_id'].astype(str)
df_sample['article_id']=["0"+x for x in df_sample['article_id']]

/var/folders/9c/xy4p20v91hv95n64yflzlj4c0000gn/T/ipykernel_77559/4080146942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['article_id'] = df_sample['article_id'].astype(str)
/var/folders/9c/xy4p20v91hv95n64yflzlj4c0000gn/T/ipykernel_77559/4080146942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['article_id']=["0"+x for x in df_sample['article_id']]


## Text and Image Embedding

In [17]:
import os
from PIL import Image
import numpy as np
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [18]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [19]:
class image_title_dataset():
    def __init__(self, text_paths, list_image_path):
        self.text_paths = text_paths
        self.list_image_path = list_image_path

    def __len__(self):
        return len(self.text_paths)

    def __getitem__(self, idx):
        return self.list_image_path[idx], self.text_paths[idx]

In [20]:
'/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/'+ df_sample.iloc[0,0]+'.jpg'

'/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/0108775015.jpg'

In [21]:
df_sample['article_id'] = df_sample['article_id'].apply(lambda x : '/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/'+ x+'.jpg')
df_sample.head()

/var/folders/9c/xy4p20v91hv95n64yflzlj4c0000gn/T/ipykernel_77559/260621178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['article_id'] = df_sample['article_id'].apply(lambda x : '/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/'+ x+'.jpg')


,article_id,caption
0,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Vest top, Jersey top with n..."
1,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid White Vest top, Jersey top with n..."
2,/Users/nikhilkudupudi/Projects - School & Work...,"womens Stripe Off White Vest top, Jersey top w..."
3,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Leggings, Tights with built..."
4,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Dark Blue Top, Fitted top in soft..."


In [22]:
from sklearn.model_selection import train_test_split

In [26]:
train,test = train_test_split(df_sample, test_size=0.2, random_state= 42)
train.reset_index(inplace= True,drop= True)
test.reset_index(inplace= True,drop= True)

In [29]:
print(len(train))
print(len(test))

44265
11067


In [31]:
train_dataloader = DataLoader(image_title_dataset(train['caption'],train['article_id']), batch_size = 32, shuffle= True)
val_dataloader = DataLoader(image_title_dataset(test['caption'],test['article_id']), batch_size= 32,shuffle= True)

In [68]:
test_data

,index,article_id,caption
0,25,/Users/nikhilkudupudi/Projects - School & Work...,"mens Melange Dark Grey Sweater, Long-sleeved s..."
1,26,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Leggings, Semi shiny stay-u..."
2,27,/Users/nikhilkudupudi/Projects - School & Work...,"mens Solid Black Trousers, Running trousers in..."
3,28,/Users/nikhilkudupudi/Projects - School & Work...,"mens Solid Dark Grey Trousers, Running trouser..."
4,29,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Beige Leggings, Matt tights with ..."


In [32]:
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

In [33]:
# !pip install clip
import clip 

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

num_epochs = 10
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images,texts = batch 
        images = [Image.open(img_path) for img_path in images]
        inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation= True, max_length = 77)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Forward pass
        outputs = model(**inputs)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text
        
        # Compute loss
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text
        
        # Backward pass
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()

        
        pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")

    # Validation phase
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # No gradients needed for validation
        for batch in val_dataloader:
            images, texts = batch
            images = [Image.open(img_path) for img_path in images]
            inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation= True, max_length = 77)
            inputs = {key: value.to(device) for key, value in inputs.items()}

            outputs = model(**inputs)
            logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            val_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")


Epoch 0/10, Loss: 3.1953:   0%|                | 3/1384 [00:06<52:11,  2.27s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/0578319001.jpg'

# testing 

In [54]:
# Assuming val_dataloader is defined and contains the validation dataset
for epoch in range(num_epochs):
    # Training phase
    model.train()  # Set model to training mode
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images, texts = batch
        images = [Image.open(img_path) for img_path in images]
        inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        outputs = model(**inputs)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

        ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
        total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

        total_loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")

    # Validation phase
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # No gradients needed for validation
        for batch in val_dataloader:
            images, texts = batch
            images = [Image.open(img_path) for img_path in images]
            inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
            inputs = {key: value.to(device) for key, value in inputs.items()}

            outputs = model(**inputs)
            logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            val_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")


Epoch 0/10, Loss: 0.3488: 100%|███████████████████| 3/3 [00:03<00:00,  1.24s/it]


KeyError: 3